In [1]:
import sys
project_path = r"d:/Download_Github_Desktop/Khoa_Luan_Tot_Nghiep_Agentic_RAG/"
sys.path.append(project_path)
import json 
from source.function.utils_shared import clean_code_fence_safe,fix_json_string
from source.model.extract_model import Bert_Extract
from source.model.generate_model import Gemini
from source.extract.utils_extract import Extract_Information
from source.core.config import Settings
from source.generate.generate import Gemini_Generate
from source.tool.google_search import GoogleSearchTool
from source.core.config import Settings

In [2]:
query="bao nhiêu tuổi được phép đăng kí kết hôn ?"

In [3]:
setting=Settings()

In [4]:
google_search_tools=GoogleSearchTool(setting)

In [5]:
result_link=google_search_tools.search(query)

> list các đường link  -> 5 link

In [6]:
print(result_link)

['https://luatsudian.com/do-tuoi-dang-ky-ket-hon.html', 'https://thuvienphapluat.vn/phap-luat/bao-nhieu-tuoi-thi-het-duoc-dang-ky-ket-hon-nhung-hanh-vi-nao-bi-cam-de-bao-ve-hon-nhan-gia-dinh-806660-164967.html', 'https://hoaithanh-hoainhon.binhdinh.gov.vn/tim-hieu-kien-thuc-phap-luat/luat-hon-nhan-va-gia-dinh2/do-tuoi-ket-hon-theo-quy-dinh-cua-phap-luat-la-bao-nhieu-.html', 'https://thuvienphapluat.vn/chinh-sach-phap-luat-moi/vn/ho-tro-phap-luat/tu-van-phap-luat/61514/huong-dan-xac-dinh-do-tuoi-dang-ky-ket-hon-cua-nam-va-nu-moi-nhat', 'https://plo.vn/16-tuoi-mang-thai-co-dang-ky-ket-hon-duoc-khong-post761656.html']


> list các nội dung trang web (5 nội dung)

In [7]:
result_docs=google_search_tools.extract_texts_from_links(result_link)

d:\Download_Python\lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'luatsudian.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\Download_Python\lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'thuvienphapluat.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\Download_Python\lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'thuvienphapluat.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\Download_Python\lib\site-packages\urllib3\connectionpool.py:1103: In

In [8]:
print(len(result_docs))

5


> Đưa qua Bert 

> tạo model bert và truyền vào để lấy hàm extract thông tin

In [9]:
bert_model=Bert_Extract(setting)
bert_utils=Extract_Information(bert_model)

config.json:   0%|          | 0.00/853 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

> trả ra dưới dạng  {key:value} ở đây key là thứ tự document,value là text sau khi được rút

In [10]:
result_reduce=bert_utils.predict(result_docs,query)

Quy_định Độ tuổi đăng_ký_kết_hôn của pháp_luật Mới 2025 Skip to content Tư_vấn miễn_phí . Hỗ_trợ nhanh , hiệu_quả Tư_vấn miễn_phí . Hỗ_trợ nhanh , hiệu_quả Trang_Chủ Giới_Thiệu Liên_Hệ Tư_Vấn Pháp_Luật Tư_Vấn Luật Hôn_Nhân Gia_Đình Tư_Vấn Luật Doanh_Nghiệp Tư_Vấn Luật Bảo_Hiểm_Xã_Hội Tư_Vấn Luật Cho Việt_Kiều Người Nước_Ngoài Tư_Vấn Luật Dân_Sự Tư_Vấn Luật Đất_Đai – Nhà ở Tư_Vấn Luật Hình_Sự Tư_Vấn Luật Lao_Động Tư_Vấn Luật Thừa_kế Biểu_Mẫu Tin_Tức Trang_Chủ » Tư_Vấn Pháp_Luật » Tư_Vấn Luật Hôn_Nhân Gia_Đình Quy_định Độ tuổi đăng_ký_kết_hôn của pháp_luật Mới 2025 Cập_nhật : Tháng_Một 7 , 2025 Tháng Ba 2 , 2025 bởi Tán Lê_Thảo_Duyên Mục_Lục Quy_định về độ tuổi đăng_ký_kết_hôn của nam và nữ ở Việt_Nam Hướng_dẫn_xác_định độ tuổi đăng_ký_kết_hôn của nam và nữ Cập_nhật 2025 Quy_định về xử_lý việc kết_hôn trái pháp_luật Giải_đáp một_số câu hỏi thường gặp về độ tuổi kết_hôn Tại_sao nam 20 tuổi mới được kết_hôn ? Bao_nhiêu tuổi được đăng_ký_kết_hôn ? Kết_hôn dưới 18 tuổi phạt bao_nhiêu nam tù 

In [11]:
print(result_reduce)

{0: 'Quy định Độ tuổi đăng ký kết hôn của pháp luật Mới 2025 Skip to content Tư vấn miễn phí. Hỗ trợ nhanh, hiệu quả Tư vấn miễn phí. Hỗ trợ nhanh, hiệu quả Trang Chủ Giới Thiệu Liên Hệ Tư Vấn Pháp Luật Tư Vấn Luật Hôn Nhân Gia Đình Tư Vấn Luật Doanh Nghiệp Tư Vấn Luật Bảo Hiểm Xã Hội Tư Vấn Luật Cho Việt Kiều Người Nước Ngoài Tư Vấn Luật Dân Sự Tư Vấn Luật Đất Đai – Nhà ở Tư Vấn Luật Hình Sự Tư Vấn Luật Lao Động Tư Vấn Luật Thừa kế Biểu Mẫu Tin Tức Trang Chủ » Tư Vấn Pháp Luật » Tư Vấn Luật Hôn Nhân Gia Đình Quy định Độ tuổi đăng ký kết hôn của pháp luật Mới 2025 Cập nhật: Tháng Một 7, 2025 Tháng Ba 2, 2025 bởi Tán Lê Thảo Duyên Mục Lục Quy định về độ tuổi đăng ký kết hôn của nam và nữ ở Việt Nam Hướng dẫn xác định độ tuổi đăng ký kết hôn của nam và nữ Cập nhật 2025 Quy định về xử lý việc kết hôn trái pháp luật Giải đáp một số câu hỏi thường gặp về độ tuổi kết hôn Tại sao nam 20 tuổi mới được kết hôn? Bao nhiêu tuổi được đăng ký kết hôn? Kết hôn dưới 18 tuổi phạt bao nhiêu nam tù? Lấy

> truyền ngữ cảnh đã trích xuất vào gemini

In [12]:
gemini=Gemini(setting)
gemini_func=Gemini_Generate(gemini,setting)

In [13]:
result_final=gemini_func.generate_response(query,result_reduce)

Failed to batch ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')
post: trace=9b41c593-259d-4cbe-9ee0-319ca0501c36,id=9b41c593-259d-4cbe-9ee0-319ca0501c36; trace=9b41c593-259d-4cbe-9ee0-319ca0501c36,id=ec4b672e-de68-445b-9c14-31c91bdeb0dc
Failed to batch ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')
post: trace=9b41c593-259d-4cbe-9ee0-319ca0501c36,id=1633cc6a-4bb4-4931-b7c7-0c63a1965a21; patch: trace=9b41c593-259d-4cbe-9ee0-319c

In [14]:
print(result_final)

```json
{
   "answer": "Độ tuổi được phép đăng ký kết hôn được quy định như sau:\n\n*   Nam giới phải từ đủ 20 tuổi trở lên.\n*   Nữ giới phải từ đủ 18 tuổi trở lên.\n\nĐộ tuổi này được tính dựa trên ngày, tháng và năm sinh.\n\nVậy, nam từ đủ 20 tuổi và nữ từ đủ 18 tuổi trở lên thì được phép đăng ký kết hôn.",
   "key": [
    0,
    3
   ]
}
```


> gọi 3 hàm này để xử lý

In [15]:
answer_result= clean_code_fence_safe(result_final)
answer_result= fix_json_string(answer_result)
answer_result= json.loads(answer_result)

In [16]:
print(answer_result)

{'answer': 'Độ tuổi được phép đăng ký kết hôn được quy định như sau:\n\n*   Nam giới phải từ đủ 20 tuổi trở lên.\n*   Nữ giới phải từ đủ 18 tuổi trở lên.\n\nĐộ tuổi này được tính dựa trên ngày, tháng và năm sinh.\n\nVậy, nam từ đủ 20 tuổi và nữ từ đủ 18 tuổi trở lên thì được phép đăng ký kết hôn.', 'key': [0, 3]}


In [17]:
print(answer_result['answer'])

Độ tuổi được phép đăng ký kết hôn được quy định như sau:

*   Nam giới phải từ đủ 20 tuổi trở lên.
*   Nữ giới phải từ đủ 18 tuổi trở lên.

Độ tuổi này được tính dựa trên ngày, tháng và năm sinh.

Vậy, nam từ đủ 20 tuổi và nữ từ đủ 18 tuổi trở lên thì được phép đăng ký kết hôn.


> key này mapping qua link để sắp xếp thứ tự theo cái này , key này nó lấy link 1 nên mapping qua cái list link ban đầu để lấy đường link, rồi hiển thị ưu tiên đầu tiên.Gemini đâu vào là 5 link nhưng việc nó chọn link nào là do bản thân nó.Nói chung tui viết oke hết rồi ô xử lý logic đoạn FE với BE xíu 

In [18]:
print(answer_result['key'])

[0, 3]
